In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

#### Funciones

In [2]:
def doClean(text):   
    
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', text)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = a.lstrip()
    # Quitar espacio derecha
    a = a.rstrip()
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

#### Carga y union de ficheros

In [3]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [4]:
os.listdir()

['sociedad_seguridad_guardia_civil_2019_limpio.csv',
 'limpieza_universidades.ipynb',
 'union_ficheros.ipynb',
 'wine-prediction-Plantilla.ipynb',
 'sociedad_educacion_universidad_2018_limpio.csv',
 'sociedad_sanidad_hospitales_2019_limpio.csv',
 '222450592859675.ipynb',
 'sociedad_educacion_colegios_2020_limpio.csv',
 'limpieza_establecimientos.ipynb',
 '.ipynb_checkpoints',
 'limpieza_hospitales.ipynb',
 'limpieza_usos_suelo.ipynb',
 'limpieza_bibliotecas.ipynb',
 'Challenge_CityRentals.ipynb',
 'tratamiento',
 'limpieza_guardia_civil.ipynb',
 'fichero_municipios.ipynb',
 'sociedad_usos_suelo_2019_limpio.csv',
 'economia_turismo_establecimientos_20??_limpio.csv',
 'manual.txt',
 'sociedad_cultura_bibliotecas_2017_limipio.csv',
 'limpieza_colegios.ipynb',
 'sociedad_educacion_nivel_formacion_ccaa_2018.csv']

## Fichero maestro

In [20]:
maestro_municipios = pd.read_csv("../maestro_idiomas.csv", index_col=0, dtype={'postal_code': object})

In [21]:
maestro_municipios.shape

(10696, 16)

In [22]:
maestro_municipios.head()

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
0,01001,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco
1,01002,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco
2,01003,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco
3,01004,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco
4,01005,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco


In [23]:
maestro_municipios[maestro_municipios['cities_es']=='segorbe']

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
2075,12400,segorbe,segorbe,castellon,castellon,comunidad valenciana,comunidad valenciana,segorbe/castellon,segorbe/castellon,segorbe/comunidad valenciana,segorbe/comunidad valenciana,segorbe/castellon,castellon/comunidad valenciana,castellon/comunidad valenciana,castellon/comunidad valenciana,castellon/comunidad valenciana


## Ficheros especificos

##### Bibliotecas

In [92]:
bibliotecas = pd.read_csv("sociedad_cultura_bibliotecas_2017_limipio.csv", index_col=0)

In [93]:
bibliotecas.shape

(3323, 4)

In [94]:
bibliotecas.head(2)

,municipio_limpio,ccaa,provincia,total_bibliotecas
0,alaior,illes_balears,illes_balears,1
1,alaro,illes_balears,illes_balears,1


In [95]:
bibliotecas['cities_es/ccaa'] = bibliotecas['municipio_limpio']+"/"+bibliotecas['ccaa']

In [96]:
bibliotecas[bibliotecas['municipio_limpio']=='segorbe']

,municipio_limpio,ccaa,provincia,total_bibliotecas,cities_es/ccaa
3313,segorbe,comunidad_valenciana,castellon_castello,2,segorbe/comunidad_valenciana


In [97]:
union = pd.merge(maestro_municipios, bibliotecas, left_on='cities_es/ccaa_es', right_on='cities_es/ccaa', how='left')

In [98]:
union[~union['total_bibliotecas'].isnull()].shape

(2421, 21)

In [99]:
union.head(2)

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa_x,cities_es/province_es,cities_es/province,cities_es/ccaa_es,...,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,municipio_limpio,ccaa_y,provincia,total_bibliotecas,cities_es/ccaa_y
0,01001,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco,NaN,NaN,NaN,NaN,NaN
1,01002,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco,NaN,NaN,NaN,NaN,NaN


In [100]:
union = union.drop(columns=['municipio_limpio', 'ccaa_y', 'cities_es/ccaa_y'])

In [144]:
union.shape

(10696, 23)

##### Hospitales

In [101]:
hospitales = pd.read_csv("sociedad_sanidad_hospitales_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [102]:
hospitales.shape

(613, 2)

In [103]:
hospitales.head(2)

,cp,uds_hospiatales
0,01002,1
1,01006,1


In [104]:
hospitales[hospitales['cp']=='12400']

,cp,uds_hospiatales


In [105]:
union = pd.merge(union, hospitales, left_on='postal_code', right_on='cp', how='left')

In [106]:
union[~union['uds_hospiatales'].isnull()].shape

(595, 20)

In [107]:
union.head(2)

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa_x,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa_x,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa,provincia,total_bibliotecas,cp,uds_hospiatales
0,01001,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco,NaN,NaN,NaN,NaN
1,01002,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,vitoria-gasteiz/pais vasco,vitoria-gasteiz/alava,alava/pais vasco,alava/pais vasco,araba/pais vasco,araba/pais vasco,NaN,NaN,01002,1.0


In [108]:
union = union.drop(columns=['cp'])

In [144]:
union.shape

(10696, 23)

##### Colegios

In [109]:
colegios = pd.read_csv("sociedad_educacion_colegios_2020_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [110]:
colegios.shape

(7838, 5)

In [111]:
colegios.head(2)

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
0,00000,a_coruna,a_coruna,6.0,NaN
1,00000,a_pontenova,lugo,NaN,1.0


In [112]:
union = pd.merge(union, colegios, left_on='postal_code', right_on='codigo_postal', how='left')

In [113]:
union[~union['num_colegios_publicos'].isnull()].shape

(7041, 24)

In [116]:
union.columns

Index(['postal_code', 'cities_es', 'cities', 'province_es', 'province',
       'ccaa_es', 'ccaa_x', 'cities_es/province_es', 'cities_es/province',
       'cities_es/ccaa_es', 'cities_es/ccaa_x', 'cities/province_es',
       'province_es/ccaa_es', 'province_es/ccaa', 'province/ccaa_es',
       'province/ccaa', 'provincia_x', 'total_bibliotecas', 'uds_hospiatales',
       'codigo_postal', 'municipio_limpio', 'provincia_y',
       'num_colegios_privados', 'num_colegios_publicos'],
      dtype='object')

In [117]:
union = union.drop(columns=['municipio_limpio', 'provincia_y', 'provincia_x'])

In [144]:
union.shape

(10696, 23)

##### Guardia Civil

In [118]:
guardia_civil = pd.read_csv("sociedad_seguridad_guardia_civil_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [119]:
guardia_civil.shape

(2237, 4)

In [120]:
guardia_civil.head(2)

,cp,municipio_limpio,provincia,uds_guardia_civil
0,10003,caceres,caceres,1
1,1010,vitoria,alava,1


In [124]:
guardia_civil['cities_es/province_es'] = guardia_civil['municipio_limpio']+"/"+guardia_civil['provincia']

In [125]:
guardia_civil[guardia_civil['municipio_limpio']=='segorbe']

,cp,municipio_limpio,provincia,uds_guardia_civil,cities_es/ccaa,cities_es/province_es
143,12400,segorbe,castellon,1,segorbe/castellon,segorbe/castellon


In [127]:
union = pd.merge(union, guardia_civil, left_on='cities_es/province_es', right_on='cities_es/province_es', how='left')

In [128]:
union[~union['uds_guardia_civil'].isnull()].shape

(5103, 26)

In [129]:
union.head(2)

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa_x,cities_es/province_es,cities_es/province,cities_es/ccaa_es,...,total_bibliotecas,uds_hospiatales,codigo_postal,num_colegios_privados,num_colegios_publicos,cp,municipio_limpio,provincia,uds_guardia_civil,cities_es/ccaa
0,01001,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,NaN,NaN,01001,1.0,3.0,NaN,NaN,NaN,NaN,NaN
1,01002,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,NaN,1.0,01002,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [130]:
union = union.drop(columns=['municipio_limpio', 'provincia', 'cities_es/ccaa', 'cp'])

In [144]:
union.shape

(10696, 23)

##### Universidades

In [137]:
universidad = pd.read_csv('sociedad_educacion_universidad_2018_limpio.csv', index_col=0, dtype={'cp': object})

In [138]:
universidad.shape

(165, 6)

In [139]:
universidad.head(2)

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0
1,alcala_de_henares,madrid,1.0,1.0,5.0,114.0


In [140]:
universidad['cities_es/ccaa'] = universidad['municipio_limpio']+"/"+universidad['ccaa']

In [141]:
universidad[universidad['municipio_limpio']=='segorbe']

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas,cities_es/ccaa


In [135]:
union = pd.merge(maestro_municipios, universidad, left_on='cities_es/ccaa_es', right_on='cities_es/ccaa', how='left')

In [142]:
union[~union['num_universidades_publicas'].isnull()].shape

(491, 23)

In [143]:
union.head(2)

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa_x,cities_es/province_es,cities_es/province,cities_es/ccaa_es,...,province_es/ccaa,province/ccaa_es,province/ccaa,municipio_limpio,ccaa_y,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas,cities_es/ccaa_y
0,01001,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,alava/pais vasco,araba/pais vasco,araba/pais vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,alava/pais vasco,araba/pais vasco,araba/pais vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
union = union.drop(columns=['municipio_limpio', 'ccaa_y', 'cities_es/ccaa_y'])

In [144]:
union.shape

(10696, 23)

##### Establecimientos

In [145]:
establecimientos = pd.read_csv("economia_turismo_establecimientos_20??_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [146]:
establecimientos.shape

(2063, 35)

In [147]:
establecimientos.head(2)

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,00000,aldea_de_cuenca_hinojares,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000,arroyo_frio_la_iruela,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [149]:
establecimientos[establecimientos['codigo_postal']=='12400']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
692,12400,segorbe,comunitat_valenciana,1.0,NaN,4.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN


In [151]:
union = pd.merge(union, establecimientos, left_on='postal_code', right_on='codigo_postal', how='left')

In [152]:
union[~union['agencias_de_viajes'].isnull()].shape

(149, 58)

In [155]:
union.columns

Index(['postal_code', 'cities_es', 'cities', 'province_es', 'province',
       'ccaa_es', 'ccaa_x', 'cities_es/province_es', 'cities_es/province',
       'cities_es/ccaa_es', 'cities_es/ccaa_x', 'cities/province_es',
       'province_es/ccaa_es', 'province_es/ccaa', 'province/ccaa_es',
       'province/ccaa', 'municipio_limpio_x', 'ccaa_y',
       'num_universidades_privadas', 'num_universidades_publicas',
       'num_tit_uni_privadas', 'num_tit_uni_publicas', 'cities_es/ccaa_y',
       'codigo_postal', 'municipio_limpio_y', 'ccaa', 'agencias_de_viajes',
       'albergues', 'alojamientos_rurales', 'artesanos',
       'atraque_de_cruceros_y_ferries', 'bares_y_cafeterías', 'bodegas',
       'campings', 'campos_de_golf_y_campos_de_pitch&putt', 'comercios',
       'convention_bureaux',
       'empresas_de_alquiler_de_vehículos_y_embarcaciones',
       'empresas_de_transporte_turístico', 'empresas_de_turismo_activo',
       'escuelas_de_español_para_extranjeros', 'espacios_naturales_protegi

In [108]:
union = union.drop(columns=['cp'])

In [144]:
union.shape

(10696, 23)

##### Usos suelo

In [157]:
usos_suelo = pd.read_csv("sociedad_usos_suelo_2019_limpio.csv", index_col=0)

In [158]:
usos_suelo.shape

(7610, 12)

In [159]:
usos_suelo.head(2)

,municipio_limpio,provincia,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos
0,abengibre,albacete,"302,079",13,3,2,1,169,1,2,1,0
1,alatoz,albacete,"339,890",41,2,1,1,140,4,2,2,0


In [160]:
usos_suelo['cities_es/province_es'] = usos_suelo['municipio_limpio']+"/"+usos_suelo['provincia']

In [161]:
usos_suelo[usos_suelo['municipio_limpio']=='segorbe']

,municipio_limpio,provincia,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos,cities_es/province_es
1855,segorbe,castellon,"2,448,844","3,314",322,12,15,378,26,89,11,19,segorbe/castellon


In [162]:
union = pd.merge(union, usos_suelo, left_on='cities_es/province_es', right_on='cities_es/province_es', how='left')

In [164]:
union[~union['uds_hosteleria'].isnull()].shape

(5872, 70)

In [165]:
union.head(2)

,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa_x,cities_es/province_es,cities_es/province,cities_es/ccaa_es,...,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos
0,01001,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,vitoria-gasteiz,vitoria-gasteiz,alava,araba,pais vasco,pais vasco,vitoria-gasteiz/alava,vitoria-gasteiz/araba,vitoria-gasteiz/pais vasco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
union = union.drop(columns=['municipio_limpio', 'provincia', 'cities_es/ccaa', 'cp'])

In [144]:
union.shape

(10696, 23)

In [166]:
import sys
print(sys.version)

3.8.2 (default, Mar 26 2020, 15:53:00) 
[GCC 7.3.0]
